## Problem Set #7

MACS 30100, Dr. Evans
<br>
Due Monday, Mar. 5 at 11:30am
<br>
Liqiang Yu

In [29]:
import pandas as pd

auto = pd.read_csv('Auto.csv',na_values='?')

In [30]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


### Problem 1
Classiﬁer “horse” race (10 points).

(a) Use sklearn.linear model.LogisticRegression to ﬁt a logistic model of mpg high on features number of cylinders (cyl), engine displacement (dspl), horsepower (hpwr), vehicle weight (wgt), acceleration (accl), vehicle year (yr), vehicle origin (orgn). Make sure to include a constant term. Fit the model using k-fold cross validation with k = 4 folds. 

In [31]:
auto.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight            int64
acceleration    float64
year              int64
origin            int64
name             object
dtype: object

In [32]:
auto = auto.dropna()
auto['mpg_high'] = auto['mpg'].apply(lambda x: 1 if x >= auto['mpg'].median() else 0)

In [33]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

Xvars = auto[['cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
yvals = auto['mpg_high'].values

k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvars)
Logit_err_0 = np.zeros(k)
Logit_err_1 = np.zeros(k)
Logit_MSE_VEC = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    print('K index: ', k_ind + 1)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    err = y_test != y_pred
    Logit_MSE_VEC[k_ind] = err.mean()
    Logit_err_0[k_ind] = ((y_pred == 0) * err).sum() / (y_pred == 0).sum() 
    Logit_err_1[k_ind] = ((y_pred == 1) * err).sum() / (y_pred == 1).sum() 
    print(classification_report(y_test, y_pred))
    print('The error rate(category 0) is', Logit_err_0[k_ind])
    print('The error rate(category 1) is', Logit_err_1[k_ind])
    print('The MSE is', Logit_MSE_VEC[k_ind])
    k_ind += 1
print('\n k-Fold Reults:')
print('The average error rate (mpg_high = 0) is', Logit_err_0.mean())
print('The average error rate (mpg_high = 1) is', Logit_err_1.mean())
print('The average MSE is: ', Logit_MSE_VEC.mean())

K index:  1
             precision    recall  f1-score   support

          0       0.94      0.89      0.92        55
          1       0.87      0.93      0.90        43

avg / total       0.91      0.91      0.91        98

The error rate(category 0) is 0.057692307692307696
The error rate(category 1) is 0.13043478260869565
The MSE is 0.09183673469387756
K index:  2
             precision    recall  f1-score   support

          0       0.88      0.91      0.90        47
          1       0.92      0.88      0.90        51

avg / total       0.90      0.90      0.90        98

The error rate(category 0) is 0.12244897959183673
The error rate(category 1) is 0.08163265306122448
The MSE is 0.10204081632653061
K index:  3
             precision    recall  f1-score   support

          0       0.85      0.87      0.86        45
          1       0.88      0.87      0.88        53

avg / total       0.87      0.87      0.87        98

The error rate(category 0) is 0.15217391304347827
The er

(b) Use sklearn.ensemble.RandomForestClassifier to ﬁt a random forest model of mpg high on max features=2 out of the seven possible features used in part (a). Set n estimators=20, set bootstrap=True, set oob score=True, and set random state=25. Report the MSE of the random forest model as the MSE from the .oob prediction object, and report the error rates for each category of mpg high from the .oob prediction object.

In [34]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,oob_score=True, random_state=25)
RF.fit(Xvars, yvals)
oob_prediction = RF.oob_decision_function_.T[1]
MSE_RF = pd.DataFrame({'pred' : oob_prediction, 'yvals': yvals})
MSE_RF['pred'] = MSE_RF['pred'].apply(lambda x: 1 if x >= 0.5 else 0)

error = (MSE_RF['pred'] != MSE_RF['yvals']).apply(int)
RF_err_0 = ((MSE_RF['pred'] == 0) * error).sum() / (MSE_RF['pred'] == 0).sum() 
RF_err_1 = ((MSE_RF['pred'] == 1) * error).sum() / (MSE_RF['pred'] == 1).sum() 

print(classification_report(MSE_RF['yvals'], MSE_RF['pred']))
print('The error rate (mpg_high = 0) is', RF_err_0.mean())
print('The error rate (mpg_high = 1) is', RF_err_1.mean())
print('The MSE of the model is', error.mean())

             precision    recall  f1-score   support

          0       0.94      0.91      0.93       196
          1       0.92      0.94      0.93       196

avg / total       0.93      0.93      0.93       392

The error rate (mpg_high = 0) is 0.05789473684210526
The error rate (mpg_high = 1) is 0.08415841584158416
The MSE of the model is 0.07142857142857142


(c) Use sklearn.svm.SVC to ﬁt a support vector machines model of mpg high with a Gaussian radial basis function kernel kernel=’rbf’ on the seven features used in part (a). Set the penalty parameter to C=1 and set gamma=0.2. Fit the model using k-fold cross validation with k = 4 folds exactly as in part (a).

In [35]:
from sklearn import svm

SVC_err_0 = np.zeros(k)
SVC_err_1 = np.zeros(k)
SVC_MSE_VEC = np.zeros(k)
k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    print('K index: ', k_ind + 1)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svc = svm.SVC(kernel='rbf', gamma=0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    err = y_test != y_pred
    SVC_MSE_VEC[k_ind] = err.mean()
    SVC_err_0[k_ind] = ((y_pred == 0) * err).sum() / (y_pred == 0).sum() 
    SVC_err_1[k_ind] = ((y_pred == 1) * err).sum() / (y_pred == 1).sum() 
    print(classification_report(y_test, y_pred, digits=3))
    print('The error rate(mpg_high = 0) is', SVC_err_0[k_ind])
    print('The error rate(mpg_high = 1) is', SVC_err_1[k_ind])
    print('The MSE is', SVC_MSE_VEC[k_ind])
    k_ind += 1
print('\nk-Fold Reults:')
SVC_err_1 = SVC_err_1[~np.isnan(SVC_err_1)]
print('The average error rate(category 0) is', SVC_err_0.mean())
print('The average error rate(category 1) is', SVC_err_1.mean())
print('The average MSE of the model is', SVC_MSE_VEC.mean())

K index:  1
             precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

The error rate(mpg_high = 0) is 0.0
The error rate(mpg_high = 1) is 0.5520833333333334
The MSE is 0.5408163265306123
K index:  2
             precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

The error rate(mpg_high = 0) is 0.5204081632653061
The error rate(mpg_high = 1) is nan
The MSE is 0.5204081632653061
K index:  3
             precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

The error rate(mpg_high = 0) is 0.53125
The error rate(mpg_high = 1) is 0.0
The MSE i

/Users/Rex/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in long_scalars
/Users/Rex/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(d) Which of the above three models do you think is the best predictor of mpg high? Why?

The average MSE for Logistic Regression is:  0.10714285714285715
<br>
The average MSE for SVM is:  0.07142857142857142
<br>
The average MSE for Random Forest is:  0.5076530612244898

By comparing the average MSE across Logistic Regression, SVM and Random Forest, we conclude that Random Forest gives the best result in terms of average MSE.
<br>
However, we notice that in SVM and Logistic Regression, they used the same k-fold data set, but we used Bootstrap and OOB to calculate the MSE for Random Forest. 

According to https://stats.stackexchange.com/questions/206807/are-k-fold-cross-validation-bootstrap-out-of-bag-fundamentally-same :

K-Fold Cross Validation (CV)

It consists in dividing the original set of observations into k subset of more or less same size. Then, you will use one of the subset as test set and the remaining subsets will be used to form your training set. You will repeat this kth times, where each time the subset used as test set will change. As an example, if you use 3-fold CV, your original set will be divided into k1, k2, k3. First, k1 will form the test set, k2 and k3 will form the training set. Then, k2 will form the test set, k1 and k3 will form the training set. Finally, k3 will form the test set, k1 and k3 will form the training set. For each fold, you output the results and you aggregate these to obtain the final result.

Bootstrap

A bootstrap is a random subset of your original data, sometimes drawn with replacement (check http://www.stat.washington.edu/courses/stat527/s13/readings/EfronTibshirani_JASA_1997.pdf on the .632 rule), sometimes not. But the idea is that a bootstrap contains only a part of your whole set of observations. It is different from CV as it does not contain a testing set. Bootstrap is used to train a different classifier each time on a different set of observations. To output your results, a combination method is used, like averaging for example.

Out-of-bag

As said above, not all observations are used to form bootstrap. The part not used forms the out-of-bag classifier, and can be used to assess the error rate of your classifier. Out-of-bag are typically used to compute the error-rate, and not to train your classifier.

And https://stats.stackexchange.com/questions/18348/differences-between-cross-validation-and-bootstrapping-to-estimate-the-predictio : 

"It comes down to variance and bias (as usual). CV tends to be less biased but K-fold CV has fairly large variance. On the other hand, bootstrapping tends to drastically reduce the variance but gives more biased results (they tend to be pessimistic). Other bootstrapping methods have been adapted to deal with the bootstrap bias (such as the 632 and 632+ rules).

Two other approaches would be "Monte Carlo CV" aka "leave-group-out CV" which does many random splits of the data (sort of like mini-training and test splits). Variance is very low for this method and the bias isn't too bad if the percentage of data in the hold-out is low. Also, repeated CV does K-fold several times and averages the results similar to regular K-fold. I'm most partial to this since it keeps the low bias and reduces the variance."

So bootstrapping tends not to overfit the data and thus tends to give a higher MSE. So we believe that Random Forest is the best model among all three models.